# Description

(Please, take a look at the README.md file in this directory for instructions on how to run this notebook)

This notebook compiles information about the GWAS and TWAS for a particular cohort. For example, the set of GWAS variants, variance of predicted expression of genes, etc.

It has specicfic parameters for papermill (see under `Settings` below).

This notebook is not directly run. See README.md.

# Modules

In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
from pathlib import Path
import pickle

import numpy as np
import pandas as pd

import conf
from entity import Gene

# Settings

In [3]:
# a cohort name (it could be something like UK_BIOBANK, etc)
COHORT_NAME = None

# reference panel such as 1000G or GTEX_V8
REFERENCE_PANEL = "GTEX_V8"

# predictions models such as MASHR or ELASTIC_NET
EQTL_MODEL = "MASHR"

# a string with a path pointing to an imputed GWAS
GWAS_FILE = None

# a string with a path pointing where S-PrediXcan results (tissue-specific are located
SPREDIXCAN_FOLDER = None

# an f-string with one placeholder {tissue}
SPREDIXCAN_FILE_PATTERN = None

# a string with a path pointing to an S-MultiXcan result
SMULTIXCAN_FILE = None

# output dir
OUTPUT_DIR_BASE = None

In [4]:
# Parameters
COHORT_NAME = "copd_only"
GWAS_FILE = "/opt/data/projects/asthma-copd/results/final_imputed_gwas/GWAS_COPD_only_GLM_SNPs_info0.7-harmonized-imputed.txt.gz"
SPREDIXCAN_FOLDER = "/opt/data/projects/asthma-copd/results/twas/spredixcan"
SPREDIXCAN_FILE_PATTERN = "GWAS_COPD_only_GLM_SNPs_info0.7-gtex_v8-mashr-{tissue}.csv"
SMULTIXCAN_FILE = "/opt/data/projects/asthma-copd/results/twas/smultixcan/GWAS_COPD_only_GLM_SNPs_info0.7-gtex_v8-mashr-smultixcan.txt"
OUTPUT_DIR_BASE = "/opt/data/projects/asthma-copd/results/gls_phenoplier"


In [5]:
assert COHORT_NAME is not None and len(COHORT_NAME) > 0, "A cohort name must be given"

COHORT_NAME = COHORT_NAME.lower()
display(f"Cohort name: {COHORT_NAME}")

'Cohort name: copd_only'

In [6]:
assert (
    REFERENCE_PANEL is not None and len(REFERENCE_PANEL) > 0
), "A reference panel must be given"

display(f"Reference panel: {REFERENCE_PANEL}")

'Reference panel: GTEX_V8'

In [7]:
assert GWAS_FILE is not None and len(GWAS_FILE) > 0, "A GWAS file path must be given"
GWAS_FILE = Path(GWAS_FILE).resolve()
assert GWAS_FILE.exists(), "GWAS file does not exist"

display(f"GWAS file path: {str(GWAS_FILE)}")

'GWAS file path: /opt/data/projects/asthma-copd/results/final_imputed_gwas/GWAS_COPD_only_GLM_SNPs_info0.7-harmonized-imputed.txt.gz'

In [8]:
assert (
    SPREDIXCAN_FOLDER is not None and len(SPREDIXCAN_FOLDER) > 0
), "An S-PrediXcan folder path must be given"
SPREDIXCAN_FOLDER = Path(SPREDIXCAN_FOLDER).resolve()
assert SPREDIXCAN_FOLDER.exists(), f"S-PrediXcan folder does not exist: {str(SPREDIXCAN_FOLDER)}"

display(f"S-PrediXcan folder path: {str(SPREDIXCAN_FOLDER)}")

'S-PrediXcan folder path: /opt/data/projects/asthma-copd/results/twas/spredixcan'

In [9]:
assert (
    SPREDIXCAN_FILE_PATTERN is not None and len(SPREDIXCAN_FILE_PATTERN) > 0
), "An S-PrediXcan file pattern must be given"
assert (
    "{tissue}" in SPREDIXCAN_FILE_PATTERN
), "S-PrediXcan file pattern must have a '{tissue}' placeholder"

display(f"S-PrediXcan file template: {SPREDIXCAN_FILE_PATTERN}")

'S-PrediXcan file template: GWAS_COPD_only_GLM_SNPs_info0.7-gtex_v8-mashr-{tissue}.csv'

In [10]:
assert (
    SMULTIXCAN_FILE is not None and len(SMULTIXCAN_FILE) > 0
), "An S-MultiXcan result file path must be given"
SMULTIXCAN_FILE = Path(SMULTIXCAN_FILE).resolve()
assert SMULTIXCAN_FILE.exists(), f"S-MultiXcan result file does not exist: {str(SMULTIXCAN_FILE)}"

display(f"S-MultiXcan file path: {str(SMULTIXCAN_FILE)}")

'S-MultiXcan file path: /opt/data/projects/asthma-copd/results/twas/smultixcan/GWAS_COPD_only_GLM_SNPs_info0.7-gtex_v8-mashr-smultixcan.txt'

In [11]:
assert (
    EQTL_MODEL is not None and len(EQTL_MODEL) > 0
), "A prediction/eQTL model must be given"

display(f"eQTL model: {EQTL_MODEL}")

'eQTL model: MASHR'

In [12]:
assert (
    OUTPUT_DIR_BASE is not None and len(OUTPUT_DIR_BASE) > 0
), "Output directory path must be given"

OUTPUT_DIR_BASE = (
    Path(OUTPUT_DIR_BASE)
    / "gene_corrs"
    / COHORT_NAME
    # / REFERENCE_PANEL.lower()
    # / EQTL_MODEL.lower()
).resolve()

OUTPUT_DIR_BASE.mkdir(parents=True, exist_ok=True)

display(f"Using output dir base: {OUTPUT_DIR_BASE}")

'Using output dir base: /opt/data/projects/asthma-copd/results/gls_phenoplier/gene_corrs/copd_only'

In [13]:
# OUTPUT_DIR_BASE = (
#     conf.RESULTS["GLS"]
#     / "gene_corrs"
#     / "cohorts"
#     / COHORT_NAME
#     / REFERENCE_PANEL.lower()
#     / EQTL_MODEL.lower()
# )

# OUTPUT_DIR_BASE.mkdir(parents=True, exist_ok=True)

# display(f"Using output dir base: {OUTPUT_DIR_BASE}")

# Load MultiPLIER Z genes

In [14]:
multiplier_z_genes = pd.read_pickle(
    conf.MULTIPLIER["MODEL_Z_MATRIX_FILE"]
).index.tolist()

In [15]:
len(multiplier_z_genes)

6750

In [16]:
assert len(multiplier_z_genes) == len(set(multiplier_z_genes))

In [17]:
multiplier_z_genes[:5]

['GAS6', 'MMP14', 'DSP', 'MARCKSL1', 'SPARC']

# GWAS

In [18]:
gwas_file_columns = pd.read_csv(GWAS_FILE, sep="\t", nrows=2).columns

assert (
    "panel_variant_id" in gwas_file_columns
), "The GWAS file must be the final imputed one using the TWAS imputation tools with column 'panel_variant_id'"

assert (
    "pvalue" in gwas_file_columns
), "The GWAS file must be the final imputed one using the TWAS imputation tools with column 'pvalue'"

assert (
    "zscore" in gwas_file_columns
), "The GWAS file must be the final imputed one using the TWAS imputation tools with column 'zscore'"

In [19]:
gwas_data = pd.read_csv(
    GWAS_FILE,
    sep="\t",
    usecols=["panel_variant_id", "pvalue", "zscore"],
)

In [20]:
gwas_data.shape

(8590917, 3)

In [21]:
gwas_data.head()

,panel_variant_id,zscore,pvalue
0,chr1_54490_G_A_b38,0.399701,0.689377
1,chr1_87021_T_C_b38,1.185593,0.235783
2,chr1_263722_C_G_b38,0.288888,0.772667
3,chr1_594402_C_T_b38,0.346451,0.729003
4,chr1_630555_C_T_b38,0.091724,0.926917


In [22]:
gwas_data.dropna().shape

(8586183, 3)

In [23]:
# remove SNPs with no results
gwas_data = gwas_data.dropna()

In [24]:
gwas_data.shape

(8586183, 3)

## Save GWAS variants

In [25]:
gwas_data.head()

,panel_variant_id,zscore,pvalue
0,chr1_54490_G_A_b38,0.399701,0.689377
1,chr1_87021_T_C_b38,1.185593,0.235783
2,chr1_263722_C_G_b38,0.288888,0.772667
3,chr1_594402_C_T_b38,0.346451,0.729003
4,chr1_630555_C_T_b38,0.091724,0.926917


In [26]:
# in eMERGE's results, some values here are repeated (will be removed later by taking the unique set of variant IDs).
gwas_data["panel_variant_id"].is_unique

True

In [27]:
gwas_variants_ids_set = frozenset(gwas_data["panel_variant_id"])
list(gwas_variants_ids_set)[:5]

['chr7_153471808_G_T_b38',
 'chr14_77634994_G_C_b38',
 'chr16_63261216_G_A_b38',
 'chr12_72010714_A_C_b38',
 'chr12_125510320_G_A_b38']

In [28]:
with open(OUTPUT_DIR_BASE / "gwas_variant_ids.pkl", "wb") as handle:
    pickle.dump(gwas_variants_ids_set, handle, protocol=pickle.HIGHEST_PROTOCOL)

# TWAS

## Available tissues for eQTL model

In [29]:
prediction_model_tissues = conf.PHENOMEXCAN["PREDICTION_MODELS"][
    f"{EQTL_MODEL}_TISSUES"
].split(" ")

In [30]:
len(prediction_model_tissues)

49

In [31]:
prediction_model_tissues[:5]

['Thyroid',
 'Artery_Aorta',
 'Heart_Atrial_Appendage',
 'Liver',
 'Heart_Left_Ventricle']

## S-MultiXcan results

In [32]:
smultixcan_results = pd.read_csv(
    SMULTIXCAN_FILE, sep="\t", usecols=["gene", "gene_name", "pvalue", "n", "n_indep"]
)

In [33]:
smultixcan_results.shape

(22329, 5)

In [34]:
smultixcan_results = smultixcan_results.dropna()

In [35]:
smultixcan_results.shape

(22326, 5)

In [36]:
smultixcan_results = smultixcan_results.assign(
    gene_id=smultixcan_results["gene"].apply(lambda g: g.split(".")[0])
)

In [37]:
smultixcan_results.head()

,gene,gene_name,pvalue,n,n_indep,gene_id
0,ENSG00000041357.15,PSMA4,1.536671e-22,49.0,4.0,ENSG00000041357
1,ENSG00000168743.12,NPNT,1.710790e-22,49.0,5.0,ENSG00000168743
2,ENSG00000080644.15,CHRNA3,2.757816e-21,19.0,4.0,ENSG00000080644
3,ENSG00000169684.13,CHRNA5,1.915449e-20,46.0,3.0,ENSG00000169684
4,ENSG00000188266.13,HYKK,7.494625e-20,34.0,5.0,ENSG00000188266


In [38]:
assert smultixcan_results["gene_id"].is_unique

### Get common genes with MultiPLIER

In [39]:
common_genes = set(multiplier_z_genes).intersection(
    set(smultixcan_results["gene_name"])
)

In [40]:
len(common_genes)

6445

In [41]:
sorted(list(common_genes))[:5]

['A2M', 'AAAS', 'AANAT', 'AARS', 'AARS2']

## Genes info

In [42]:
multiplier_gene_obj = {
    gene_name: Gene(name=gene_name)
    for gene_name in common_genes
    if gene_name in Gene.GENE_NAME_TO_ID_MAP
}

In [43]:
# delete common_genes, from now on, genes_info should be used for common genes
del common_genes

In [44]:
len(multiplier_gene_obj)

6445

In [45]:
assert multiplier_gene_obj["GAS6"].ensembl_id == "ENSG00000183087"

In [46]:
_gene_obj = list(multiplier_gene_obj.values())

genes_info = pd.DataFrame(
    {
        "name": [g.name for g in _gene_obj],
        "id": [g.ensembl_id for g in _gene_obj],
        "chr": [g.chromosome for g in _gene_obj],
        "band": [g.band for g in _gene_obj],
        "start_position": [g.get_attribute("start_position") for g in _gene_obj],
        "end_position": [g.get_attribute("end_position") for g in _gene_obj],
    }
)

In [47]:
genes_info = genes_info.assign(
    gene_length=genes_info.apply(
        lambda x: x["end_position"] - x["start_position"], axis=1
    )
)

In [48]:
genes_info.dtypes

name               object
id                 object
chr                object
band               object
start_position    float64
end_position      float64
gene_length       float64
dtype: object

In [49]:
_tmp = genes_info[genes_info.isna().any(axis=1)]
display(_tmp)
assert _tmp.shape[0] < 5

,name,id,chr,band,start_position,end_position,gene_length
1189,TMEM133,ENSG00000170647,None,None,NaN,NaN,NaN
3426,TBCE,ENSG00000116957,None,None,NaN,NaN,NaN


In [50]:
genes_info = genes_info.dropna()

In [51]:
genes_info["chr"] = genes_info["chr"].apply(pd.to_numeric, downcast="integer")
genes_info["start_position"] = genes_info["start_position"].astype(int)
genes_info["end_position"] = genes_info["end_position"].astype(int)
genes_info["gene_length"] = genes_info["gene_length"].astype(int)

In [52]:
genes_info.dtypes

name              object
id                object
chr                 int8
band              object
start_position     int64
end_position       int64
gene_length        int64
dtype: object

In [53]:
assert genes_info["name"].is_unique

In [54]:
assert genes_info["id"].is_unique

In [55]:
genes_info.shape

(6443, 7)

In [56]:
genes_info.head()

,name,id,chr,band,start_position,end_position,gene_length
0,PPP1R13B,ENSG00000088808,14,14q32.33,103733195,103847590,114395
1,HNRNPA0,ENSG00000177733,5,5q31.2,137745651,137754376,8725
2,GSN,ENSG00000148180,9,9q33.2,121207794,121332843,125049
3,CCR3,ENSG00000183625,3,3p21.31,46163604,46266706,103102
4,TRIB2,ENSG00000071575,2,2p24.3,12716889,12742734,25845


In [57]:
genes_info.sort_values("chr")

,name,id,chr,band,start_position,end_position,gene_length
2186,BCL10,ENSG00000142867,1,1p22.3,85265776,85276904,11128
5913,C4BPA,ENSG00000123838,1,1q32.2,207104233,207144972,40739
3442,GNPAT,ENSG00000116906,1,1q42.2,231241207,231277973,36766
4823,SLAMF8,ENSG00000158714,1,1q23.2,159826811,159837492,10681
1215,MRPL24,ENSG00000143314,1,1q23.1,156737303,156741590,4287
...,...,...,...,...,...,...,...
4003,ALG12,ENSG00000182858,22,22q13.33,49900229,49918438,18209
6323,SHANK3,ENSG00000251322,22,22q13.33,50674415,50733298,58883
289,GAL3ST1,ENSG00000128242,22,22q12.2,30554635,30574665,20030
2762,FBXO7,ENSG00000100225,22,22q12.3,32474676,32498829,24153


### Save

In [58]:
genes_info.to_pickle(OUTPUT_DIR_BASE / "genes_info.pkl")

## S-PrediXcan results

### Load results across all tissues

In [59]:
spredixcan_result_files = {
    t: SPREDIXCAN_FOLDER / SPREDIXCAN_FILE_PATTERN.format(tissue=t)
    for t in prediction_model_tissues
}

In [60]:
assert len(spredixcan_result_files) == len(prediction_model_tissues)
display(list(spredixcan_result_files.values())[:5])

[PosixPath('/opt/data/projects/asthma-copd/results/twas/spredixcan/GWAS_COPD_only_GLM_SNPs_info0.7-gtex_v8-mashr-Thyroid.csv'),
 PosixPath('/opt/data/projects/asthma-copd/results/twas/spredixcan/GWAS_COPD_only_GLM_SNPs_info0.7-gtex_v8-mashr-Artery_Aorta.csv'),
 PosixPath('/opt/data/projects/asthma-copd/results/twas/spredixcan/GWAS_COPD_only_GLM_SNPs_info0.7-gtex_v8-mashr-Heart_Atrial_Appendage.csv'),
 PosixPath('/opt/data/projects/asthma-copd/results/twas/spredixcan/GWAS_COPD_only_GLM_SNPs_info0.7-gtex_v8-mashr-Liver.csv'),
 PosixPath('/opt/data/projects/asthma-copd/results/twas/spredixcan/GWAS_COPD_only_GLM_SNPs_info0.7-gtex_v8-mashr-Heart_Left_Ventricle.csv')]

In [61]:
# look at the structure of one result
pd.read_csv(spredixcan_result_files["Whole_Blood"]).head()

,gene,gene_name,zscore,effect_size,pvalue,var_g,pred_perf_r2,pred_perf_pval,pred_perf_qval,n_snps_used,n_snps_in_cov,n_snps_in_model,best_gwas_p,largest_weight
0,ENSG00000188266.13,HYKK,-7.977829,NaN,1.489290e-15,0.002663,NaN,NaN,NaN,1,1,1,1.489290e-15,0.076877
1,ENSG00000041357.15,PSMA4,7.580813,NaN,3.433968e-14,0.006043,NaN,NaN,NaN,3,3,3,1.884797e-15,0.161176
2,ENSG00000183049.12,CAMK1D,6.846095,NaN,7.589322e-12,0.059839,NaN,NaN,NaN,2,2,2,2.814849e-11,0.277173
3,ENSG00000165609.12,NUDT5,5.826954,NaN,5.644804e-09,0.005001,NaN,NaN,NaN,1,1,1,5.644804e-09,0.102943
4,ENSG00000198624.12,CCDC69,-5.352389,NaN,8.680055e-08,0.028808,NaN,NaN,NaN,4,4,4,1.032419e-07,0.170673


In [62]:
assert all(f.exists() for f in spredixcan_result_files.values())

In [63]:
spredixcan_dfs = [
    pd.read_csv(
        f,
        usecols=[
            "gene",
            "zscore",
            "pvalue",
            "n_snps_used",
            "n_snps_in_model",
        ],
    )
    .dropna(subset=["gene", "zscore", "pvalue"])
    .assign(tissue=t)
    for t, f in spredixcan_result_files.items()
]

In [64]:
assert len(spredixcan_dfs) == len(prediction_model_tissues)

In [65]:
spredixcan_dfs = pd.concat(spredixcan_dfs)

In [66]:
assert spredixcan_dfs["tissue"].unique().shape[0] == len(prediction_model_tissues)

In [67]:
spredixcan_dfs.shape

(659087, 6)

In [68]:
spredixcan_dfs = spredixcan_dfs.assign(
    gene_id=spredixcan_dfs["gene"].apply(lambda g: g.split(".")[0])
)

In [69]:
spredixcan_dfs.head()

,gene,zscore,pvalue,n_snps_used,n_snps_in_model,tissue,gene_id
0,ENSG00000204385.10,5.449554,5.049626e-08,3,3,Thyroid,ENSG00000204385
1,ENSG00000135074.15,5.281442,1.281709e-07,2,2,Thyroid,ENSG00000135074
2,ENSG00000244731.7,-4.999106,5.759674e-07,2,2,Thyroid,ENSG00000244731
3,ENSG00000166822.12,-4.994474,5.899636e-07,1,1,Thyroid,ENSG00000166822
4,ENSG00000168769.13,4.899470,9.609559e-07,1,1,Thyroid,ENSG00000168769


In [70]:
# leave only common genes
spredixcan_dfs = spredixcan_dfs[spredixcan_dfs["gene_id"].isin(set(genes_info["id"]))]

In [71]:
spredixcan_dfs.shape

(235749, 7)

### Count number of tissues available per gene

In [72]:
spredixcan_genes_n_models = spredixcan_dfs.groupby("gene_id")["tissue"].nunique()

In [73]:
spredixcan_genes_n_models

gene_id
ENSG00000000419     2
ENSG00000000938    36
ENSG00000000971    34
ENSG00000001084    32
ENSG00000001167    40
                   ..
ENSG00000278540    36
ENSG00000278828     4
ENSG00000278845    49
ENSG00000281005    49
ENSG00000282608    36
Name: tissue, Length: 6443, dtype: int64

In [74]:
# testing that in S-MultiXcan I get the same number of tissues per gene
_tmp_smultixcan_results_n_models = (
    smultixcan_results.set_index("gene_id")["n"].astype(int).rename("tissue")
)

_cg = _tmp_smultixcan_results_n_models.index.intersection(
    spredixcan_genes_n_models.index
)
_tmp_smultixcan_results_n_models = _tmp_smultixcan_results_n_models.loc[_cg]
_spredixcan = spredixcan_genes_n_models.loc[_cg]

assert _spredixcan.shape[0] == _tmp_smultixcan_results_n_models.shape[0]
assert _spredixcan.equals(_tmp_smultixcan_results_n_models.loc[_spredixcan.index])

### Get tissues available per gene

In [75]:
spredixcan_genes_models = spredixcan_dfs.groupby("gene_id")["tissue"].apply(
    lambda x: frozenset(x.tolist())
)

In [76]:
spredixcan_genes_models

gene_id
ENSG00000000419         (Brain_Hypothalamus, Brain_Substantia_nigra)
ENSG00000000938    (Brain_Frontal_Cortex_BA9, Nerve_Tibial, Brain...
ENSG00000000971    (Brain_Frontal_Cortex_BA9, Nerve_Tibial, Brain...
ENSG00000001084    (Brain_Frontal_Cortex_BA9, Nerve_Tibial, Brain...
ENSG00000001167    (Nerve_Tibial, Brain_Putamen_basal_ganglia, Br...
                                         ...                        
ENSG00000278540    (Brain_Frontal_Cortex_BA9, Nerve_Tibial, Brain...
ENSG00000278828    (Esophagus_Muscularis, Artery_Coronary, Cells_...
ENSG00000278845    (Brain_Frontal_Cortex_BA9, Nerve_Tibial, Brain...
ENSG00000281005    (Brain_Frontal_Cortex_BA9, Nerve_Tibial, Brain...
ENSG00000282608    (Brain_Frontal_Cortex_BA9, Nerve_Tibial, Brain...
Name: tissue, Length: 6443, dtype: object

In [77]:
assert spredixcan_genes_n_models.shape[0] == spredixcan_genes_models.shape[0]

In [78]:
assert spredixcan_genes_n_models.index.equals(spredixcan_genes_models.index)

In [79]:
assert (spredixcan_genes_models.apply(len) <= len(prediction_model_tissues)).all()

In [80]:
spredixcan_genes_models.apply(len).describe()

count    6443.000000
mean       36.589943
std        12.865559
min         1.000000
25%        29.000000
50%        42.000000
75%        47.000000
max        49.000000
Name: tissue, dtype: float64

In [81]:
# testing that I obtained the right number of tissues
assert (
    spredixcan_genes_models.loc[spredixcan_genes_n_models.index]
    .apply(len)
    .equals(spredixcan_genes_n_models)
)

### Add gene name and set index

In [82]:
spredixcan_genes_models = spredixcan_genes_models.to_frame().reset_index()

In [83]:
spredixcan_genes_models.head()

,gene_id,tissue
0,ENSG00000000419,"(Brain_Hypothalamus, Brain_Substantia_nigra)"
1,ENSG00000000938,"(Brain_Frontal_Cortex_BA9, Nerve_Tibial, Brain..."
2,ENSG00000000971,"(Brain_Frontal_Cortex_BA9, Nerve_Tibial, Brain..."
3,ENSG00000001084,"(Brain_Frontal_Cortex_BA9, Nerve_Tibial, Brain..."
4,ENSG00000001167,"(Nerve_Tibial, Brain_Putamen_basal_ganglia, Br..."


In [84]:
spredixcan_genes_models = spredixcan_genes_models.assign(
    gene_name=spredixcan_genes_models["gene_id"].apply(
        lambda g: Gene.GENE_ID_TO_NAME_MAP[g]
    )
)

In [85]:
spredixcan_genes_models = spredixcan_genes_models[["gene_id", "gene_name", "tissue"]]

In [86]:
spredixcan_genes_models = spredixcan_genes_models.set_index("gene_id")

In [87]:
spredixcan_genes_models.head()

,gene_name,tissue
gene_id,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra)"
ENSG00000000938,FGR,"(Brain_Frontal_Cortex_BA9, Nerve_Tibial, Brain..."
ENSG00000000971,CFH,"(Brain_Frontal_Cortex_BA9, Nerve_Tibial, Brain..."
ENSG00000001084,GCLC,"(Brain_Frontal_Cortex_BA9, Nerve_Tibial, Brain..."
ENSG00000001167,NFYA,"(Nerve_Tibial, Brain_Putamen_basal_ganglia, Br..."


### Add number of tissues

In [88]:
spredixcan_genes_models = spredixcan_genes_models.assign(
    n_tissues=spredixcan_genes_models["tissue"].apply(len)
)

In [89]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues
gene_id,,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra)",2
ENSG00000000938,FGR,"(Brain_Frontal_Cortex_BA9, Nerve_Tibial, Brain...",36
ENSG00000000971,CFH,"(Brain_Frontal_Cortex_BA9, Nerve_Tibial, Brain...",34
ENSG00000001084,GCLC,"(Brain_Frontal_Cortex_BA9, Nerve_Tibial, Brain...",32
ENSG00000001167,NFYA,"(Nerve_Tibial, Brain_Putamen_basal_ganglia, Br...",40


### Save

Here I quickly save these results to a file, given that the next steps (covariates) are slow to compute.

In [90]:
# this is important, other scripts depend on gene_name to be unique
assert spredixcan_genes_models["gene_name"].is_unique

In [91]:
assert not spredixcan_genes_models.isna().any(None)

/tmp/ipykernel_123/1381297071.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  assert not spredixcan_genes_models.isna().any(None)


In [92]:
spredixcan_genes_models.to_pickle(OUTPUT_DIR_BASE / "gene_tissues.pkl")

## Add covariates based on S-PrediXcan results

This extend the previous file with more columns

### Get gene's objects

In [93]:
spredixcan_gene_obj = {
    gene_id: Gene(ensembl_id=gene_id) for gene_id in spredixcan_genes_models.index
}

In [94]:
len(spredixcan_gene_obj)

6443

### Add genes' variance captured by principal components

In [95]:
# def _get_gene_pc_variance(gene_row):
#     gene_id = gene_row.name
#     gene_tissues = gene_row["tissue"]
#     gene_obj = spredixcan_gene_obj[gene_id]

#     u, s, vt = gene_obj.get_tissues_correlations_svd(
#         tissues=gene_tissues,
#         snps_subset=gwas_variants_ids_set,
#         reference_panel=REFERENCE_PANEL,
#         model_type=EQTL_MODEL,
#         # use_covariance_matrix=True,
#     )

#     return s

In [96]:
# _tmp = spredixcan_genes_models.loc["ENSG00000188976"]
# _get_gene_pc_variance(_tmp)

In [97]:
# spredixcan_genes_tissues_pc_variance = spredixcan_genes_models.apply(
#     _get_gene_pc_variance, axis=1
# )

In [98]:
# spredixcan_genes_tissues_pc_variance

In [99]:
# # testing
# assert spredixcan_genes_tissues_pc_variance.loc[
#     "ENSG00000188976"
# ].sum() == pytest.approx(44.01605629086847)
# # this is using the covariance:
# # assert spredixcan_genes_tissues_pc_variance.loc["ENSG00000188976"].sum() == pytest.approx(1.1492946006449425)

In [100]:
# # add to spredixcan_genes_models
# spredixcan_genes_models = spredixcan_genes_models.join(
#     spredixcan_genes_tissues_pc_variance.rename("tissues_pc_variances")
# )

In [101]:
# spredixcan_genes_models.shape

In [102]:
# spredixcan_genes_models.head()

### Add gene variance per tissue

In [103]:
# def _get_gene_variances(gene_row):
#     gene_id = gene_row.name
#     gene_tissues = gene_row["tissue"]

#     tissue_variances = {}
#     gene_obj = spredixcan_gene_obj[gene_id]

#     for tissue in gene_tissues:
#         tissue_var = gene_obj.get_pred_expression_variance(
#             tissue=tissue,
#             reference_panel=REFERENCE_PANEL,
#             model_type=EQTL_MODEL,
#             snps_subset=gwas_variants_ids_set,
#         )

#         if tissue_var is not None:
#             tissue_variances[tissue] = tissue_var

#     return tissue_variances

In [104]:
# _tmp = spredixcan_genes_models.loc["ENSG00000000419"]
# _get_gene_variances(_tmp)

In [105]:
# spredixcan_genes_tissues_variance = spredixcan_genes_models.apply(
#     _get_gene_variances, axis=1
# )

In [106]:
# spredixcan_genes_tissues_variance

In [107]:
# # testing
# _gene_id = "ENSG00000188976"
# x = spredixcan_genes_tissues_variance.loc[_gene_id]
# # expected value obtained by sum of PCA eigenvalues on this gene's predicted expression
# assert np.sum(list(x.values())) == pytest.approx(1.2326202607409493)

In [108]:
# # testing
# spredixcan_genes_tissues_variance.loc["ENSG00000000419"]

In [109]:
# # add to spredixcan_genes_models
# spredixcan_genes_models = spredixcan_genes_models.join(
#     spredixcan_genes_tissues_variance.rename("tissues_variances")
# )

In [110]:
# spredixcan_genes_models.shape

In [111]:
# spredixcan_genes_models.head()

### Count number of SNPs predictors used across tissue models

In [112]:
spredixcan_genes_sum_of_n_snps_used = (
    spredixcan_dfs.groupby("gene_id")["n_snps_used"].sum().rename("n_snps_used_sum")
)

In [113]:
spredixcan_genes_sum_of_n_snps_used

gene_id
ENSG00000000419     2
ENSG00000000938    40
ENSG00000000971    44
ENSG00000001084    46
ENSG00000001167    47
                   ..
ENSG00000278540    44
ENSG00000278828     5
ENSG00000278845    89
ENSG00000281005    81
ENSG00000282608    40
Name: n_snps_used_sum, Length: 6443, dtype: int64

In [114]:
# add sum of snps used to spredixcan_genes_models
spredixcan_genes_models = spredixcan_genes_models.join(
    spredixcan_genes_sum_of_n_snps_used
)

In [115]:
spredixcan_genes_models.shape

(6443, 4)

In [116]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues,n_snps_used_sum
gene_id,,,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra)",2,2
ENSG00000000938,FGR,"(Brain_Frontal_Cortex_BA9, Nerve_Tibial, Brain...",36,40
ENSG00000000971,CFH,"(Brain_Frontal_Cortex_BA9, Nerve_Tibial, Brain...",34,44
ENSG00000001084,GCLC,"(Brain_Frontal_Cortex_BA9, Nerve_Tibial, Brain...",32,46
ENSG00000001167,NFYA,"(Nerve_Tibial, Brain_Putamen_basal_ganglia, Br...",40,47


### Count number of SNPs predictors in models across tissue models

In [117]:
spredixcan_genes_sum_of_n_snps_in_model = (
    spredixcan_dfs.groupby("gene_id")["n_snps_in_model"]
    .sum()
    .rename("n_snps_in_model_sum")
)

In [118]:
spredixcan_genes_sum_of_n_snps_in_model

gene_id
ENSG00000000419     2
ENSG00000000938    40
ENSG00000000971    44
ENSG00000001084    46
ENSG00000001167    48
                   ..
ENSG00000278540    44
ENSG00000278828     5
ENSG00000278845    91
ENSG00000281005    81
ENSG00000282608    40
Name: n_snps_in_model_sum, Length: 6443, dtype: int64

In [119]:
# add sum of snps in model to spredixcan_genes_models
spredixcan_genes_models = spredixcan_genes_models.join(
    spredixcan_genes_sum_of_n_snps_in_model
)

In [120]:
spredixcan_genes_models.shape

(6443, 5)

In [121]:
spredixcan_genes_models.head()

,gene_name,tissue,n_tissues,n_snps_used_sum,n_snps_in_model_sum
gene_id,,,,,
ENSG00000000419,DPM1,"(Brain_Hypothalamus, Brain_Substantia_nigra)",2,2,2
ENSG00000000938,FGR,"(Brain_Frontal_Cortex_BA9, Nerve_Tibial, Brain...",36,40,40
ENSG00000000971,CFH,"(Brain_Frontal_Cortex_BA9, Nerve_Tibial, Brain...",34,44,44
ENSG00000001084,GCLC,"(Brain_Frontal_Cortex_BA9, Nerve_Tibial, Brain...",32,46,46
ENSG00000001167,NFYA,"(Nerve_Tibial, Brain_Putamen_basal_ganglia, Br...",40,47,48


### Summarize prediction models for each gene

In [122]:
# def _summarize_gene_models(gene_id):
#     """
#     For a given gene ID, it returns a dataframe with predictor SNPs in rows and tissues in columns, where
#     values are the weights of SNPs in those tissues.
#     It can contain NaNs.
#     """
#     gene_obj = spredixcan_gene_obj[gene_id]
#     gene_tissues = spredixcan_genes_models.loc[gene_id, "tissue"]

#     gene_models = {}
#     gene_unique_snps = set()
#     for t in gene_tissues:
#         gene_model = gene_obj.get_prediction_weights(tissue=t, model_type=EQTL_MODEL)
#         gene_models[t] = gene_model

#         gene_unique_snps.update(set(gene_model.index))

#     df = pd.DataFrame(
#         data=np.nan, index=list(gene_unique_snps), columns=list(gene_tissues)
#     )

#     for t in df.columns:
#         for snp in df.index:
#             gene_model = gene_models[t]

#             if snp in gene_model.index:
#                 df.loc[snp, t] = gene_model.loc[snp]

#     return df

In [123]:
# # testing
# spredixcan_gene_obj["ENSG00000000419"].get_prediction_weights(
#     tissue="Brain_Hypothalamus", model_type=EQTL_MODEL
# )

In [124]:
# spredixcan_gene_obj["ENSG00000000419"].get_prediction_weights(
#     tissue="Brain_Substantia_nigra", model_type=EQTL_MODEL
# )

In [125]:
# # testing
# _gene_id = "ENSG00000000419"

# _gene_model = _summarize_gene_models(_gene_id)
# assert (
#     _gene_model.loc["chr20_50862947_C_T_b38", "Brain_Hypothalamus"].round(5) == 0.43138
# )
# assert pd.isnull(_gene_model.loc["chr20_50957480_C_T_b38", "Brain_Hypothalamus"])

# assert pd.isnull(_gene_model.loc["chr20_50862947_C_T_b38", "Brain_Substantia_nigra"])
# assert (
#     _gene_model.loc["chr20_50957480_C_T_b38", "Brain_Substantia_nigra"].round(5)
#     == -0.1468
# )

In [126]:
# gene_models = {}

# for gene_id in spredixcan_genes_models.index:
#     gene_models[gene_id] = _summarize_gene_models(gene_id)

In [127]:
# # testing
# _gene_id = "ENSG00000000419"

# _gene_model = gene_models[_gene_id]
# assert (
#     _gene_model.loc["chr20_50862947_C_T_b38", "Brain_Hypothalamus"].round(5) == 0.43138
# )
# assert pd.isnull(_gene_model.loc["chr20_50957480_C_T_b38", "Brain_Hypothalamus"])

# assert pd.isnull(_gene_model.loc["chr20_50862947_C_T_b38", "Brain_Substantia_nigra"])
# assert (
#     _gene_model.loc["chr20_50957480_C_T_b38", "Brain_Substantia_nigra"].round(5)
#     == -0.1468
# )

In [128]:
# # save
# import gzip

# with gzip.GzipFile(OUTPUT_DIR_BASE / "gene_tissues_models.pkl.gz", "w") as f:
#     pickle.dump(gene_models, f)

In [129]:
# # testing saved file
# with gzip.GzipFile(OUTPUT_DIR_BASE / "gene_tissues_models.pkl.gz", "r") as f:
#     _tmp = pickle.load(f)

In [130]:
# assert len(gene_models) == len(_tmp)
# assert gene_models["ENSG00000000419"].equals(_tmp["ENSG00000000419"])

### Count number of _unique_ SNPs predictors used and available across tissue models

In [131]:
# def _count_unique_snps(gene_id):
#     """
#     For a gene_id, it counts unique SNPs in all models and their intersection with GWAS SNPs (therefore, used by S-PrediXcan).
#     """
#     gene_tissues = spredixcan_genes_models.loc[gene_id, "tissue"]

#     gene_unique_snps = set()
#     for t in gene_tissues:
#         t_snps = set(gene_models[gene_id].index)
#         gene_unique_snps.update(t_snps)

#     gene_unique_snps_in_gwas = gwas_variants_ids_set.intersection(gene_unique_snps)

#     return pd.Series(
#         {
#             "unique_n_snps_in_model": len(gene_unique_snps),
#             "unique_n_snps_used": len(gene_unique_snps_in_gwas),
#         }
#     )

In [132]:
# # testing
# spredixcan_genes_models[spredixcan_genes_models["n_snps_used_sum"] == 2].head()

In [133]:
# # case with two snps, not repeated across tissues
# _gene_id = "ENSG00000000419"
# display(
#     spredixcan_gene_obj[_gene_id].get_prediction_weights(
#         tissue="Brain_Hypothalamus", model_type=EQTL_MODEL
#     )
# )
# display(
#     spredixcan_gene_obj[_gene_id].get_prediction_weights(
#         tissue="Brain_Substantia_nigra", model_type=EQTL_MODEL
#     )
# )

In [134]:
# _tmp = _count_unique_snps(_gene_id)
# assert _tmp.shape[0] == 2
# assert _tmp["unique_n_snps_in_model"] == 2
# assert _tmp["unique_n_snps_used"] == 2

In [135]:
# # get unique snps for all genes
# spredixcan_genes_unique_n_snps = spredixcan_genes_models.groupby("gene_id").apply(
#     lambda x: _count_unique_snps(x.name)
# )

In [136]:
# spredixcan_genes_unique_n_snps.head()

In [137]:
# assert (
#     spredixcan_genes_unique_n_snps["unique_n_snps_in_model"]
#     >= spredixcan_genes_unique_n_snps["unique_n_snps_used"]
# ).all()

In [138]:
# # add unique snps to spredixcan_genes_models
# spredixcan_genes_models = spredixcan_genes_models.join(spredixcan_genes_unique_n_snps)

In [139]:
# spredixcan_genes_models.shape

In [140]:
# spredixcan_genes_models.head()

### Save

In [141]:
# this is important, other scripts depend on gene_name to be unique
assert spredixcan_genes_models["gene_name"].is_unique

In [142]:
assert not spredixcan_genes_models.isna().any(None)

/tmp/ipykernel_123/1381297071.py:1: FutureWarning: In a future version of pandas all arguments of DataFrame.any and Series.any will be keyword-only.
  assert not spredixcan_genes_models.isna().any(None)


In [143]:
spredixcan_genes_models.to_pickle(OUTPUT_DIR_BASE / "gene_tissues.pkl")